In [56]:
import sys
sys.path.append("..")

In [57]:
label_path = '../output/test.pkl'
file = '/Users/knpob/Territory/Kolmo/code/foot-measure/data/22.obj'
is_plot = True

## Load landmarks labelling

In [58]:
import pandas as pd

df = pd.read_pickle(label_path)
df

coord                                                                x  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         5.403781   
                                                   P10       37.420189   
                                                   P11       60.707325   
                                                   P12      -13.437896   
                                                   P2        39.791413   
                                                   P3       -36.608820   
                                                   P4        60.029866   
                                                   P5       -34.857391   
                                                   P6        25.580183   
                                                   P7        25.017647   
                                                   P8        62.422166   
                                                   P9         5.118006   

coord                                                                 y  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1       -148.230339   
                                                   P10      -134.368848   
                                                   P11       -83.435721   
                                                   P12      -100.379473   
                                                   P2       -151.940277   
                                                   P3       -149.343529   
                                                   P4       -143.176643   
                                                   P5       -146.440812   
                                                   P6        -99.863963   
                                                   P7        -90.009337   
                                                   P8       -132.355347   
                                                   P9       -138.997939   

coord                                                                 z  
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -89.845244  
                                                   P10       130.528426  
                                                   P11        56.076134  
                                                   P12        73.471696  
                                                   P2        -79.201353  
                                                   P3        -43.588958  
                                                   P4        -43.305646  
                                                   P5         -9.256272  
                                                   P6          7.762573  
                                                   P7         23.855555  
                                                   P8         91.321825  
                                                   P9        106.235944

## Local frame

In [59]:
import pyvista as pv
from measure import frame, visual
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file))
axes_frame, origin = frame.estimate_foot_frame(mesh, file, df)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84d36de190_28&reconnect=auto' style='wid…

In [60]:
mesh_clip = frame.foot_clip(mesh, df, file)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84b0ea5b80_29&reconnect=auto' style='wid…

In [61]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         96.988697   
                                                   P10      -125.356467   
                                                   P11       -56.817110   
                                                   P12       -68.192168   
                                                   P2         84.578820   
                                                   P3         53.644933   
                                                   P4         46.912579   
                                                   P5         19.144668   
                                                   P6         -5.261559   
                                                   P7        -22.031736   
                                                   P8        -88.051106   
                                                   P9        -98.824266   

coord                                                                y  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         1.732191   
                                                   P10      -13.248616   
                                                   P11      -47.454658   
                                                   P12       29.177257   
                                                   P2       -31.137591   
                                                   P3        47.318385   
                                                   P4       -49.115611   
                                                   P5        48.122008   
                                                   P6       -14.980776   
                                                   P7       -14.062715   
                                                   P8       -41.470343   
                                                   P9        17.245417   

coord                                                                z  
file                                               landmark             
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        22.840222  
                                                   P10       16.990838  
                                                   P11       70.588888  
                                                   P12       60.204955  
                                                   P2        14.799827  
                                                   P3        22.687951  
                                                   P4        18.738618  
                                                   P5        22.850279  
                                                   P6        61.511911  
                                                   P7        70.157216  
                                                   P8        19.283884  
                                                   P9        17.550502

## Metrics

In [62]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [63]:
from measure import metric

results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

In [64]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/code/foot-measure/data/22.obj,222.345164,172.269046,144.501135,101.208286,96.433996,58.71576,61.511911,70.157216,70.82689,150.827971,163.766897,135.15444,148.002131


### Visualization

In [65]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, file, landmark1, landmark2, axis, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f8490894610_30&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f8490782be0_31&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84a0228a90_32&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84907d2160_33&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f849078f5b0_34&reconnect=auto' style='wid…

In [66]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, file, landmark, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84a0296970_35&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84907a4fd0_36&reconnect=auto' style='wid…

In [67]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, file, landmark_origin, landmark1, landmark2, name=name)
        scene.show()

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14580' coro=<WslinkHandler.sendWrappedMessage() done, defined at /Users/knpob/opt/anaconda3/envs/mesh4d/lib/python3.9/site-packages/wslink/backends/aiohttp/__init__.py:584> exception=ConnectionResetError('Cannot write to closing transport')>
Traceback (most recent call last):
  File "/Users/knpob/opt/anaconda3/envs/mesh4d/lib/python3.9/asyncio/tasks.py", line 256, in __step
    result = coro.send(None)
  File "/Users/knpob/opt/anaconda3/envs/mesh4d/lib/python3.9/site-packages/wslink/backends/aiohttp/__init__.py", line 645, in sendWrappedMessage
    await ws.send_str(encMsg)
  File "/Users/knpob/opt/anaconda3/envs/mesh4d/lib/python3.9/site-packages/aiohttp/web_ws.py", line 308, in send_str
    await self._writer.send(data, binary=False, compress=compress)
  File "/Users/knpob/opt/anaconda3/envs/mesh4d/lib/python3.9/site-packages/aiohttp/http_websocket.py", line 690, in send
    await self._send_frame(mess

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84b0ea4c40_37&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84907827c0_38&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84a022a790_39&reconnect=auto' style='wid…

In [68]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmarks(scene, df_local, file, mesh_local, ['P4', 'P5'], name='ABG')
    scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f84a0228a60_40&reconnect=auto' style='wid…

In [69]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, file, mesh_local, 'P6', 'x', name='IG')
    scene.show()

Widget(value="<iframe src='http://localhost:57332/index.html?ui=P_0x7f8490894b20_41&reconnect=auto' style='wid…